# **Installation and Setup**

In [1]:
!pip install -q ipython-autotime
%load_ext autotime


time: 297 µs (started: 2024-03-09 09:51:57 +00:00)


In [2]:
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"

time: 4.92 s (started: 2024-03-09 09:51:57 +00:00)


In [3]:
!pip install -qqq transformers==4.37
!pip install -qqq git+https://github.com/huggingface/datasets
!pip install -qqq git+https://github.com/huggingface/peft
!pip install -qqq git+https://github.com/huggingface/accelerate
!pip install -qqq --upgrade bitsandbytes
!pip install -qqq git+https://github.com/huggingface/trl
!pip install -qqq --upgrade safetensors
!pip install -qqq -U flash-attn
!pip install -qqq evaluate
!pip install -qqq gputil psutil
!pip install -qqq wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 27.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing meta

In [4]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
time: 6.43 s (started: 2024-03-09 09:54:07 +00:00)


In [5]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randrange

# Import datasets and related modules
import datasets
from datasets import load_dataset, Dataset, DatasetDict

# Import PyTorch and Hugging Face Transformers
import torch
import transformers
from transformers import  AutoTokenizer

# Import training and evaluation components
from transformers import TrainingArguments, Trainer, set_seed

# Import Evaluate
import evaluate

time: 7.79 s (started: 2024-03-09 09:54:14 +00:00)


In [6]:
set_seed(42)

time: 2.41 ms (started: 2024-03-09 09:54:21 +00:00)


In [7]:
library_versions = {
    "Python": sys.version.split()[0],
    "NumPy": np.__version__,
    "Pandas": pd.__version__,
    "Datasets": datasets.__version__,
    "Transformers": transformers.__version__,
    "Torch": torch.__version__,
    "Evaluate": evaluate.__version__,
}

for name, version in library_versions.items():
    print(name.rjust(15), ":", version)

         Python : 3.10.12
          NumPy : 1.25.2
         Pandas : 1.5.3
       Datasets : 2.18.1.dev0
   Transformers : 4.37.0
          Torch : 2.1.0+cu121
       Evaluate : 0.4.1
time: 906 µs (started: 2024-03-09 09:54:21 +00:00)


# **Data Preparation and Analysis**

In [8]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split = "train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

time: 15.1 s (started: 2024-03-09 09:54:21 +00:00)


In [9]:
dataset

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 26872
})

time: 2.03 ms (started: 2024-03-09 09:54:36 +00:00)


In [10]:
model_id = "HuggingFaceH4/zephyr-7b-beta"

time: 234 µs (started: 2024-03-09 09:54:36 +00:00)


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

time: 4.08 s (started: 2024-03-09 09:54:36 +00:00)


In [12]:
tokenizer.pad_token = tokenizer.eos_token

time: 383 µs (started: 2024-03-09 09:54:41 +00:00)


In [13]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='HuggingFaceH4/zephyr-7b-beta', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<unk>', '<s>', '</s>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
time: 457 µs (started: 2024-03-09 09:54:41 +00:00)


In [14]:
def process_data_sample( example):

        '''
        Helper function to process the dataset sample by adding prompt and clean if necessary.

        Args:
        example: Data sample

        Returns:
        processed_example: Data sample post processing
        '''

        processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.</s>\n<|user|>\n" + example["instruction"] + "</s>\n<|assistant|>\n" + example["response"]

        return processed_example

time: 388 µs (started: 2024-03-09 09:54:41 +00:00)


In [15]:
df = dataset.to_pandas()

time: 45.7 ms (started: 2024-03-09 09:54:41 +00:00)


In [16]:
df

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...,...,...
26867,BL,I am waiting for a rebate of {{Refund Amount}}...,REFUND,track_refund,Thank you for sharing your situation regarding...
26868,BIL,how to see if there is anything wrong with my ...,REFUND,track_refund,Ensuring the accuracy of your restitution is o...
26869,BLQZ,I'm waiting for a reimbjrsement of {{Currency ...,REFUND,track_refund,"Firstly, I genuinely understand the importance..."
26870,BL,I don't know what to do to see my reimbursemen...,REFUND,track_refund,I've understood you're unsure about how to che...


time: 193 ms (started: 2024-03-09 09:54:41 +00:00)


In [17]:
df["text"] = df[["instruction", "response"]].apply(lambda x: process_data_sample(x), axis=1)

time: 253 ms (started: 2024-03-09 09:54:41 +00:00)


In [18]:
print(df)

       flags                                        instruction category  \
0          B   question about cancelling order {{Order Number}}    ORDER   
1        BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2       BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3         BL         I need to cancel purchase {{Order Number}}    ORDER   
4      BCELN  I cannot afford this order, cancel purchase {{...    ORDER   
...      ...                                                ...      ...   
26867     BL  I am waiting for a rebate of {{Refund Amount}}...   REFUND   
26868    BIL  how to see if there is anything wrong with my ...   REFUND   
26869   BLQZ  I'm waiting for a reimbjrsement of {{Currency ...   REFUND   
26870     BL  I don't know what to do to see my reimbursemen...   REFUND   
26871     BL  I need to know if there is anything new on the...   REFUND   

             intent                                           response  \
0      cancel

In [19]:
processed_data = Dataset.from_pandas(df[["text"]])

time: 169 ms (started: 2024-03-09 09:54:41 +00:00)


In [20]:
print(processed_data)

Dataset({
    features: ['text'],
    num_rows: 26872
})
time: 481 µs (started: 2024-03-09 09:54:41 +00:00)


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments

time: 273 µs (started: 2024-03-09 09:54:41 +00:00)


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

time: 2.26 ms (started: 2024-03-09 09:54:41 +00:00)


In [23]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            use_flash_attention_2=True
                                                    )

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

time: 8min 44s (started: 2024-03-09 09:54:41 +00:00)


In [24]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

time: 356 µs (started: 2024-03-09 10:03:26 +00:00)


In [25]:
model = prepare_model_for_kbit_training(model)

time: 29.5 ms (started: 2024-03-09 10:03:26 +00:00)


In [26]:
print("\n====================================================================\n")
print("\t\t\tMODEL CONFIG UPDATED")
print("\n====================================================================\n")



			MODEL CONFIG UPDATED


time: 555 µs (started: 2024-03-09 10:03:26 +00:00)


In [27]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNor

In [28]:
print(model.config)

MistralConfig {
  "_name_or_path": "HuggingFaceH4/zephyr-7b-beta",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformer

In [29]:
peft_config = LoraConfig(
                                    r=64,
                                    lora_alpha=64,
                                    lora_dropout=0.0,
                                    bias="none",
                                    task_type="CAUSAL_LM",
                                    target_modules= ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
                                )


time: 321 µs (started: 2024-03-09 10:03:26 +00:00)


In [30]:
model = get_peft_model(model, peft_config)

time: 2.01 s (started: 2024-03-09 10:03:26 +00:00)


In [31]:
print("\n====================================================================\n")
print("\t\t\tPREPARED MODEL FOR FINETUNING")
print(model)
print(model.config)
print("\n====================================================================\n")



			PREPARED MODEL FOR FINETUNING
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bi

In [32]:
print(model.print_trainable_parameters())

trainable params: 167,772,160 || all params: 7,409,504,256 || trainable%: 2.264283199029719
None
time: 5.88 ms (started: 2024-03-09 10:03:28 +00:00)


In [33]:
print("\n====================================================================\n")
print("\t\t\tPREPARED FOR FINETUNING")
print("\n====================================================================\n")



			PREPARED FOR FINETUNING


time: 393 µs (started: 2024-03-09 10:03:28 +00:00)


# **Instantiate LoRAConfig and Prepare Model for Training**

In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir= f"zephyr-7B-beta-bitext-customer-support",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=2e-4,
    report_to="wandb",
    optim="adamw_torch_fused",
    bf16=True,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_strategy='epoch', # log according to log_steps
    save_safetensors=True,
    push_to_hub=True,                      # push model to hub
    seed = 42
)

time: 1.84 ms (started: 2024-03-09 10:03:28 +00:00)


# **Initializing the SFTTrainer with Configuration for Model Training**

In [35]:
from huggingface_hub import notebook_login
notebook_login()

time: 16.6 ms (started: 2024-03-09 10:03:28 +00:00)


In [37]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=processed_data,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    args=args,
)


Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


time: 3.94 s (started: 2024-03-09 10:04:51 +00:00)


In [38]:
!pip install -qqq wandb

time: 5.32 s (started: 2024-03-09 10:04:55 +00:00)


In [39]:
import wandb

time: 297 µs (started: 2024-03-09 10:05:01 +00:00)


In [40]:
wandb.init(project="zephyr_vs_zephyrgemma_bitext_customer_support", entity="drishtisharma96505", group='zephyr')


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


time: 26.5 s (started: 2024-03-09 10:05:05 +00:00)


In [41]:
import time


time: 763 µs (started: 2024-03-09 10:05:32 +00:00)


In [42]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.73 GB of memory reserved.
time: 2.07 ms (started: 2024-03-09 10:05:32 +00:00)


In [43]:
start_time = time.time()  # Start timer

trainer_stats = trainer.train()

end_time = time.time()  # End timer
training_duration = (end_time - start_time)/60

print(f"Training Time: {training_duration} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
3359,0.518800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
3359,0.518800
6718,0.410800
10077,0.288500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Training Time: 245.41594293514888 minutes
time: 4h 5min 24s (started: 2024-03-09 10:05:32 +00:00)


In [44]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Li

In [45]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

14667.0872 seconds used for training.
244.45 minutes used for training.
Peak reserved memory = 19.795 GB.
Peak reserved memory for training = 14.065 GB.
Peak reserved memory % of max memory = 50.033 %.
Peak reserved memory for training % of max memory = 35.55 %.
time: 5.45 ms (started: 2024-03-09 14:10:57 +00:00)


# **Push to Hub**

In [46]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/DrishtiSharma/zephyr-7B-beta-bitext-customer-support/commit/b4395ef5ef8338abc7e912016cba0c172bbcd1a5', commit_message='End of training', commit_description='', oid='b4395ef5ef8338abc7e912016cba0c172bbcd1a5', pr_url=None, pr_revision=None, pr_num=None)

time: 7.24 s (started: 2024-03-09 14:10:57 +00:00)


In [47]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

def process_data_sample(example):

    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example["instruction"] + "\n<|assistant|>\n"

    return processed_example

tokenizer = AutoTokenizer.from_pretrained("zephyr-7B-beta-bitext-customer-support")

inp_str = process_data_sample(
    {
        "instruction": "i have a question about cancelling order {{Order Number}}",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "zephyr-7B-beta-bitext-customer-support",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

time: 14.8 s (started: 2024-03-09 14:25:26 +00:00)


In [48]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
i have a question about cancelling order {{Order Number}}
<|assistant|>
I've understood you have a question regarding canceling your order with the order number {{Order Number}}. I'm here to assist you and provide the information you need. Please go ahead and let me know what specific question you have, and I'll do my best to address your concerns and guide you through the cancellation process. Your satisfaction is our top priority.

Note: This conversation has been edited for clarity and brevity.
7.975819110870361
time: 7.98 s (started: 2024-03-09 14:26:06 +00:00)


In [55]:
inp_str = process_data_sample(
    {
        "instruction": "Unfortunately, I have ordered a product twice, and now I want to cancel order {{Order Number}}. Could you please help me in this regard?",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "zephyr-7B-beta-bitext-customer-support",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=400,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

time: 14.2 s (started: 2024-03-09 14:41:18 +00:00)


In [56]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
Unfortunately, I have ordered a product twice, and now I want to cancel order {{Order Number}}. Could you please help me in this regard?
<|assistant|>
I've realized that you have ordered a product twice and now you would like to cancel order {{Order Number}}. I apologize for any inconvenience this may have caused. To assist you with canceling your order, please follow these steps:

1. Sign in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate the order with the number {{Order Number}} and click on it to view the details.
4. Look for the option labeled '{{Online Order Interaction}}' and select it.
5. Follow any additional prompts or instructions to complete the cancellation process.

If you encounter any difficulties or have further question

In [51]:
inp_str = process_data_sample(
    {
        "instruction": "Hi, I want to buy a Nutribullet blender, could you please provide me with the details of Nutribullet 900 pro?",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "zephyr-7B-beta-bitext-customer-support",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

time: 14 s (started: 2024-03-09 14:29:51 +00:00)


In [52]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
Hi, I want to buy a Nutribullet blender, could you please provide me with the details of Nutribullet 900 pro?
<|assistant|>
Thank you for your interest in purchasing our Nutribullet blender! I'm here to provide you with all the details you need. The Nutribullet 900 pro is a high-performance blender that can handle large volumes and heavy-duty tasks. It features a powerful motor with a speed of 3.5 horsepower, allowing you to blend ingredients quickly and efficiently. The blender also has a large capacity of 900 grams, making it suitable for commercial or industrial use. Additionally, it comes with a range of accessories and attachments to enhance your blending experience. If you have any specific questions or need further assistance, please feel free to ask. I'm here to help you make an informed decision and ensure a smooth purchasing process. Let's get

In [57]:
inp_str = process_data_sample(
    {
        "instruction": "Hi, I want to re-order the cancelled product.",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "zephyr-7B-beta-bitext-customer-support",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

time: 13.9 s (started: 2024-03-09 14:43:53 +00:00)


In [58]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
Hi, I want to re-order the cancelled product.
<|assistant|>
I've understood, you're looking to re-order the cancelled product. I apologize for any inconvenience caused by the cancellation. To assist you further, could you please provide me with the order number or any other relevant details of the cancelled product? This will help me locate your order and guide you through the re-ordering process smoothly. Thank you for your cooperation, and I'm here to ensure a seamless experience for you. Let's get started!

Please note that the availability of the re-ordering option may depend on various factors, such as the product's availability, your location, and the specific circumstances of the cancellation. If you have any questions or need additional assistance, feel free to let me know. I'm here to help!

Update: Thank you for reaching out! I'm happy to assi

In [59]:
inp_str = process_data_sample(
    {
        "instruction": "Hi, I want to delete my Amazon account.",
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "zephyr-7B-beta-bitext-customer-support",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

time: 14 s (started: 2024-03-09 14:45:48 +00:00)


In [60]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.
<|user|>
Hi, I want to delete my Amazon account.
<|assistant|>
Thank you for bringing this to our attention. I understand that you would like to delete your Amazon account, and I'm here to assist you with that. To proceed with the account deletion, I will need some information from you. Could you please provide me with your account username or any other identifying details? This will help me locate your account and initiate the deletion process. Rest assured that your request will be handled promptly and securely. Once I have the necessary information, I will guide you through the account deletion steps. Thank you for choosing us, and I appreciate your cooperation in this matter. Let's work together to ensure a smooth account closure experience for you. Please feel free to ask any further questions or share any additional concerns you may have. We are here to su